# Data & Tools Exploration

This notebook walks you through the foundational layer of the AML investigation setup:
- What the database looks like and how it's built
- How to explore transactions and accounts manually
- What the `ReadOnlySqlDatabase` tool is and why it exists
- How an agent would "see" the database through that tool

In [ ]:
import os
import sqlite3
from pathlib import Path

import pandas as pd
from aieng.agent_evals.aml_investigation.data import download_dataset_file, normalize_transactions_data
from aieng.agent_evals.tools.sql_database import ReadOnlySqlDatabase
from dotenv import load_dotenv


# Setting the notebook directory to the project's root folder
if Path("").absolute().name == "eval-agents":
    print(f"Notebook path is already the root path: {Path('').absolute()}")
else:
    os.chdir(Path("").absolute().parent.parent)
    print(f"The notebook path has been set to: {Path('').absolute()}")

load_dotenv(verbose=True)

## 1. Exploring the dataset

We will be using the [IBM Transactions for Anti Money Laundering (AML)](https://www.kaggle.com/datasets/ealtman2019/ibm-transactions-for-anti-money-laundering-aml) dataset, which is available on Kaggle. It contains synthetic transaction data designed to mimic real-world financial transactions, including both legitimate and potentially fraudulent activities. The dataset includes various features such as transaction amount, type, origin and destination accounts, timestamps, and a label indicating whether the transaction is fraudulent or not.

### 1.1 Downloading the dataset

There are 6 datasets available, divided into two groups of three sets. The groups are based on the ratio of illicit transactions in the data:
- Group **HI** contains relatively higher illicit transaction ratios (i.e. more laundering activity)
- Group **LI** contains relatively lower illicit transaction ratios (i.e. less laundering activity)

Each group has three sets of data based on the total number of transactions/accounts: "Small", "Medium", and "Large".

You can download any of the six datasets using the `download_dataset_file` function. However, **note that the code tries to load all the data into memory at once, so the "Medium" and "Large" datasets may cause memory issues on some machines**. For this reason, we recommend starting with the "Small" dataset.

Each dataset has 3 files that you can download separately:
- `<HI/LI>_<Small/Medium/Large>_Trans.csv`: contains transaction data, with each row representing a single transaction.
- `<HI/LI>_<Small/Medium/Large>_accounts.csv`: contains account data, with each row representing a single account.
- `<HI/LI>_<Small/Medium/Large>_Patterns.txt`: contains ground-truth laundering patterns, which are groups of transactions that are known to be part of the same laundering scheme. Each pattern includes a list of transaction IDs that are involved in that pattern.

#### 1.1.1. The transactions

In [ ]:
path_to_transactions_csv = download_dataset_file(illicit_ratio="HI", transactions_size="Small", filename="Trans.csv")
print(f"Path to transactions.csv: {path_to_transactions_csv}")

In [ ]:
transactions_df = pd.read_csv(path_to_transactions_csv)
transactions_df.head(10)

In [ ]:
# Are there duplicates?
print(f"Number of duplicate transactions: {transactions_df.duplicated().sum()}")

Notice that the transactions dataset needs some cleaning. For example:
- There are duplicate transactions that should be removed before analysis.
- There are two columns that have the same name "Account". Pandas automatically renamed the second one to "Account.1", but we should rename them to something more descriptive.

We use the `normalize_transactions_data` function to perform these cleaning steps and make the transactions data easier to work with.

In [ ]:
transactions_df = normalize_transactions_data(transactions_df)
transactions_df.head(10)

#### 1.1.2. The accounts

In [ ]:
path_to_accounts_csv = download_dataset_file(illicit_ratio="HI", transactions_size="Small", filename="accounts.csv")
print(f"Path to accounts.csv: {path_to_accounts_csv}")

In [ ]:
accounts_df = pd.read_csv(path_to_accounts_csv)
accounts_df.head(10)

Similar to the transactions dataset, we can rename the columns in the accounts dataset to make them easier to work with.

In [ ]:
accounts_df.rename(
    columns={
        "Bank Name": "bank_name",
        "Bank ID": "bank_id",
        "Account Number": "account_number",
        "Entity ID": "entity_id",
        "Entity Name": "entity_name",
    },
    inplace=True,
)
accounts_df.head(10)

#### 1.1.3. The patterns

In [ ]:
path_to_patterns_txt = download_dataset_file(illicit_ratio="HI", transactions_size="Small", filename="Patterns.txt")
print(f"Path to patterns.txt: {path_to_patterns_txt}")

Laundering patterns start with `BEGIN LAUNDERING ATTEMPT` and end with `END LAUNDERING ATTEMPT`. Each pattern includes a list of transactions that are involved in that pattern.

In [ ]:
# Print the first laundering pattern
begin_prefix = "BEGIN LAUNDERING ATTEMPT - "
end_prefix = "END LAUNDERING ATTEMPT"
with open(path_to_patterns_txt, "r") as f:
    for line in f:
        if line.startswith(begin_prefix):
            print(f"\n{line.strip()}")
        elif line.startswith(end_prefix):
            print(f"{line.strip()}")
            break
        else:
            print(line.strip())

## 2. Build the Database

With the datasets downloaded and cleaned, we can build a SQLite database that contains the transactions and accounts data. This will allow us to query the data using SQL, which is a common way for agents to interact with databases.

In [ ]:
DB_PATH = Path("implementations/aml_investigation/data/aml_transactions.db")
DDL_PATH = Path("implementations/aml_investigation/data/schema.ddl")

print(f"Database exists: {DB_PATH.exists()}")
print(f"DDL file exists: {DDL_PATH.exists()}")

In [ ]:
# Run this cell only if the database doesn't exist yet.
# It will build the SQLite DB. It may take some time to run.

if not DB_PATH.exists():
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute("PRAGMA foreign_keys = ON;")

        if not DDL_PATH.exists():
            raise FileNotFoundError(f"DDL file not found at {DDL_PATH}")

        with open(DDL_PATH, "r") as file:
            conn.executescript(file.read())
        conn.commit()

        # Add new columns to the transactions DataFrame: date, day_of_week, time_of_day
        transactions_df["date"] = pd.to_datetime(transactions_df["timestamp"]).dt.date
        transactions_df["day_of_week"] = pd.to_datetime(transactions_df["timestamp"]).dt.day_name()
        transactions_df["time_of_day"] = pd.to_datetime(transactions_df["timestamp"]).dt.time

        # Set Transaction ID as index
        transactions_df.set_index("transaction_id", drop=True, inplace=True)

        # NOTE: We drop the "is_laundering" column since that's the label the
        # agent is trying to predict, and it wouldn't be present in a real
        # investigation scenario.
        transactions_df.drop(columns=["is_laundering"], inplace=True)

        accounts_df.to_sql("accounts", conn, if_exists="append", index=False)
        transactions_df.to_sql("transactions", conn, if_exists="append")
else:
    print("Database already exists — skipping creation.")

### 2.1. Understand the Schema

The database has two core tables and one convenience view:

- **`accounts`** — who owns each account (bank, account number, entity name)
- **`transactions`** — every transfer between accounts (amount, currency, timestamp, payment format)
- **`account_transactions`** (view) — a flattened, account-centric view of transactions. Each transaction appears **twice**: once as an OUT row for the sender, once as an IN row for the receiver. This makes it easy to query all activity for a single account without a `UNION` every time.

Let's look at the raw DDL:

In [ ]:
print(DDL_PATH.read_text())

## 3. Manual Exploration with `pandas` + `sqlite3`

Let's get familiar with the data before involving any agent.

In [ ]:
conn = sqlite3.connect(DB_PATH)

# Quick sanity check: how many rows in each table?
for table in ["accounts", "transactions"]:
    count = pd.read_sql(f"SELECT COUNT(*) AS n FROM {table}", conn).iloc[0]["n"]
    print(f"{table:20s}: {count:,} rows")

In [ ]:
# Preview the accounts table
pd.read_sql("SELECT * FROM accounts LIMIT 10", conn)

In [ ]:
# Preview the transactions table
pd.read_sql("SELECT * FROM transactions LIMIT 10", conn)

In [ ]:
# Preview the account_transactions view
# Notice each transaction appears as both an IN row and an OUT row
sample_tx = pd.read_sql("SELECT transaction_id FROM transactions LIMIT 1", conn).iloc[0]["transaction_id"]

print(f"Looking up transaction: {sample_tx}\n")
pd.read_sql(f"SELECT * FROM account_transactions WHERE transaction_id = '{sample_tx}'", conn)

In [ ]:
conn.close()

## 4. The `ReadOnlySqlDatabase` Tool

So far we've been using `sqlite3` directly — a regular connection that could run `DROP TABLE` or `DELETE` if we wanted. 

When an LLM agent runs SQL, we can't have it modifying data. The [`ReadOnlySqlDatabase`](https://github.com/VectorInstitute/eval-agents/blob/main/aieng-eval-agents/aieng/agent_evals/tools/sql_database.py) tool solves this with two layers of protection:

1. **AST-level enforcement** — It parses the SQL into a syntax tree using [SQLGlot](https://sqlglot.com/) and rejects any query that contains write operations (`INSERT`, `UPDATE`, `DROP`, etc.), even if hidden inside a CTE or subquery.
2. **Row limits + timeouts** — It caps results at `max_rows` (default 100) and cancels slow queries, preventing runaway costs.

The tool exposes exactly **two methods** that become the agent's "tools":
- `get_schema_info()` — returns the table/column names
- `execute(query)` — runs a SELECT and returns a markdown table string

In [ ]:
db = ReadOnlySqlDatabase(
    connection_uri=f"sqlite:///{DB_PATH}",
    agent_name="NotebookExplorer",
    max_rows=10,  # keep output short for this notebook
)

print("Tool created successfully!")
print(f"Max rows: {db.max_rows}")
print(f"Agent name: {db.agent_name}")

### 4.1. Schema Discovery

This is the first thing the agent does on every case — ask "what tables exist and what columns do they have?"

In [ ]:
schema = db.get_schema_info()
print(schema)

In [ ]:
# You can also ask for a specific table only
print(db.get_schema_info(table_names=["transactions"]))

### 4.2. Running Queries Through the Tool

Notice that the output is a **markdown table string** — not a DataFrame. This is intentional: the agent receives it as plain text in its context window.

In [ ]:
result = db.execute("SELECT * FROM accounts LIMIT 5")
print(result)

In [ ]:
# Aggregation query. This is the kind of query the agent would run
result = db.execute("""
    SELECT
        account,
        COUNT(*) AS tx_count,
        COUNT(DISTINCT counterparty) AS unique_counterparties,
        SUM(CASE WHEN direction='IN' THEN amount ELSE 0 END) AS total_in,
        SUM(CASE WHEN direction='OUT' THEN amount ELSE 0 END) AS total_out
    FROM account_transactions
    GROUP BY account
    ORDER BY tx_count DESC
    LIMIT 10
""")
print(result)

### 4.3. Safety Demo — Write Operations Are Blocked

Let's verify the protection actually works. The tool should reject any write operation.

In [ ]:
# Attempting a DELETE. This should be blocked
result = db.execute("DELETE FROM transactions WHERE 1=1")
print(result)  # Expect: "Query Error: Security Violation..."

In [ ]:
# Attempting a write hidden inside a CTE — also blocked
result = db.execute("""
    WITH cleanup AS (
        DELETE FROM transactions WHERE 1=1
    )
    SELECT * FROM accounts LIMIT 1
""")
print(result)  # Expect: "Query Error: Security Violation..."

In [ ]:
# Row limit enforcement — we set max_rows=10 above, so this won't return all rows
result = db.execute("SELECT * FROM transactions")
# Check the last line — it should say "Truncated at 10 rows"
for line in result.split("\n")[-3:]:
    print(line)

## Summary

In this notebook you've seen:

1. **The dataset** — the IBM Transactions for Anti Money Laundering (AML) dataset, its structure, and how to download and clean it.
2. **The database** — two tables (`accounts`, `transactions`) and a convenience view (`account_transactions`) storing synthetic AML transaction data.
3. **Manual exploration** — how to use `pandas` + `sqlite3` to query the data as a developer would.
4. **The `ReadOnlySqlDatabase` tool** — the safety-hardened wrapper the agent uses, with AST-level write blocking and row limits.

**Next:** In Notebook 2, we'll instantiate the AML agent, explore how to give it tasks, then inspect its reasoning and tool call trace.

In [ ]:
db.close()
print("Done!")